In [1]:
import numpy as np 
import pandas as pd 

from sklearn.feature_extraction.text import CountVectorizer
from keras.layers import Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential,clone_model
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras import optimizers
from keras.callbacks import ModelCheckpoint
import tensorflow as tf

import keras.backend as K

Using TensorFlow backend.


In [2]:
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

In [3]:
from utils import *
df = functions.parse_file(r"raw_data/EmoContext/train2.txt", "EmoContext")
df.head(5)

,id,turn1,turn2,turn3,label
0,0,dont worry ism girl,hmm how do i know if you are,whats ur name ? ?,others
1,1,when did in i?,saw many times i think -_-,no . i never saw you .,angry
2,2,by,by google chrome,where you live,others
3,3,u r ridiculous,i might be ridiculous but i am telling the tru...,u little disgusting whore,angry
4,4,just for time pass,wt do u do a a living then 4,maybe,others


In [4]:
text_data = []
for idx,row in df.iterrows():
    text_data.append("{}. {}. {}.".format(row['turn1'], row['turn2'], row['turn3']))

In [5]:
NR_WORDS = 5000
MAX_PROP_LENGTH = 100
tokenizer = Tokenizer(num_words=NR_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                   lower=True,split=' ')

tokenizer.fit_on_texts(text_data)

X_train = tokenizer.texts_to_sequences(text_data)
X_train = pad_sequences(X_train, maxlen = MAX_PROP_LENGTH)

In [6]:
def one_hot_vector(word,label=None):
    words = {"others": 0, "angry": 1, "sad":2, "happy": 3}
    if label == None:
        y = [0,0,0,0]
        y[words[word]] = 1
        return y
    if label == word:
        return [1,0]
    return [0,1]

Y_train_others = []
Y_train_angry = []
Y_train_sad = []
Y_train_happy = []

for idx,row in df.iterrows():
    Y_train_others.append(one_hot_vector(row['label'],"others"))
    Y_train_angry.append(one_hot_vector(row['label'],"angry"))
    Y_train_sad.append(one_hot_vector(row['label'],"sad"))
    Y_train_happy.append(one_hot_vector(row['label'],"happy"))

Y_train_others = np.array(Y_train_others)
Y_train_angry = np.array(Y_train_angry)
Y_train_sad = np.array(Y_train_sad)
Y_train_happy = np.array(Y_train_happy)

In [7]:
df = functions.parse_file(r"raw_data/EmoContext/devwithlabels.txt", "EmoContext")
df.head(5)

,id,turn1,turn2,turn3,label
0,0,Then dont ask me,YOURE A GUY NOT AS IF YOU WOULD UNDERSTAND,IM NOT A GUY FUCK OFF,angry
1,1,Mixed things such as??,the things you do.,Have you seen minions??,others
2,2,Today I'm very happy,and I'm happy for you ❤,I will be marry,happy
3,3,Woah bring me some,left it there oops,Brb,others
4,4,it is thooooo,I said soon master.,he is pressuring me,others


In [8]:
text_data = []
for idx,row in df.iterrows():
    text_data.append("{}. {}. {}.".format(row['turn1'], row['turn2'], row['turn3']))

In [9]:
#tokenizer = Tokenizer(num_words=NR_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
#                                   lower=True,split=' ')

#tokenizer.fit_on_texts(text_data)

X_test = tokenizer.texts_to_sequences(text_data)
X_test = pad_sequences(X_test, maxlen = MAX_PROP_LENGTH)

In [10]:
def one_hot_vector(word,label=None):
    words = {"others": 0, "angry": 1, "sad":2, "happy": 3}
    if label == None:
        y = [0,0,0,0]
        y[words[word]] = 1
        return y
    if label == word:
        return [1,0]
    return [0,1]

Y_test_others = []
Y_test_angry = []
Y_test_sad = []
Y_test_happy = []

for idx,row in df.iterrows():
    Y_test_others.append(one_hot_vector(row['label'],"others"))
    Y_test_angry.append(one_hot_vector(row['label'],"angry"))
    Y_test_sad.append(one_hot_vector(row['label'],"sad"))
    Y_test_happy.append(one_hot_vector(row['label'],"happy"))

Y_test_others = np.array(Y_test_others)
Y_test_angry = np.array(Y_test_angry)
Y_test_sad = np.array(Y_test_sad)
Y_test_happy = np.array(Y_test_happy)

In [11]:
embed_dim = 64
lstm_out = 32
batch_size = 128

adam = optimizers.Adam(lr=0.01)
rmsprop = optimizers.RMSprop(lr=0.01, rho=0.95, epsilon=None, decay=0.0)


model = Sequential()
model.add(Embedding(NR_WORDS, embed_dim,input_length = X_train.shape[1],name="boosting_embeding"))
model.add(LSTM(lstm_out))
model.add(Dense(2,activation='softmax'))

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
boosting_embeding (Embedding (None, 100, 64)           320000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 66        
Total params: 332,482
Trainable params: 332,482
Non-trainable params: 0
_________________________________________________________________


In [13]:
mdcheck = ModelCheckpoint("trained_models/best_model_val_acc{val_acc:.4f}.h5", monitor='val_f1', save_best_only=True)

In [14]:
Y_train_angry = np.array(Y_train_angry)

model_angry = clone_model(model)
model_angry.compile(loss = 'binary_crossentropy', optimizer="adam", metrics = ['accuracy', f1])
model_angry.name = "angry_model"
history = model_angry.fit(X_train, Y_train_angry,
                    validation_data=(X_test, Y_test_angry),
                    epochs=5, verbose=1, batch_size=batch_size,callbacks=[mdcheck],shuffle=True,
                    class_weight={
                        0: len(X_train) / len(np.where(Y_train_angry[:,0]==1.0)[0]),
                        1: len(X_train) / len(np.where(Y_train_angry[:,1]==1.0)[0]),
                    })

Train on 30160 samples, validate on 2755 samples
Epoch 1/5
30160/30160 [==============================] - 71s 2ms/step - loss: 0.7444 - acc: 0.8344 - f1: 0.7773 - val_loss: 0.2423 - val_acc: 0.9220 - val_f1: 0.7487
Epoch 2/5
30160/30160 [==============================] - 56s 2ms/step - loss: 0.3762 - acc: 0.9294 - f1: 0.8905 - val_loss: 0.2077 - val_acc: 0.9296 - val_f1: 0.7668
Epoch 3/5
30160/30160 [==============================] - 53s 2ms/step - loss: 0.3156 - acc: 0.9402 - f1: 0.9066 - val_loss: 0.2509 - val_acc: 0.9071 - val_f1: 0.7280
Epoch 4/5
30160/30160 [==============================] - 53s 2ms/step - loss: 0.2853 - acc: 0.9436 - f1: 0.9120 - val_loss: 0.2156 - val_acc: 0.9223 - val_f1: 0.7478
Epoch 5/5
30160/30160 [==============================] - 55s 2ms/step - loss: 0.2487 - acc: 0.9513 - f1: 0.9230 - val_loss: 0.2490 - val_acc: 0.9103 - val_f1: 0.7284


In [15]:
save_model(model_angry,indices=[{"indices":["turn1","turn2","turn3"],"NR_WORDS":NR_WORDS,"MAX_PROP_LENGTH":MAX_PROP_LENGTH}])

In [16]:
Y_train_sad = np.array(Y_train_sad)

model_sad = clone_model(model)
model_sad.compile(loss = 'binary_crossentropy', optimizer=rmsprop, metrics = ['accuracy', f1])
model_sad.name = "sad_model"
history = model_sad.fit(X_train, Y_train_sad,
                    validation_data=(X_test, Y_test_sad),
                    epochs=5, verbose=1, batch_size=batch_size,callbacks=[mdcheck],
                    class_weight={
                        0: len(X_train) / len(np.where(Y_train_sad[:,0]==1.0)[0]),
                        1: len(X_train) / len(np.where(Y_train_sad[:,1]==1.0)[0]),
                    })

Train on 30160 samples, validate on 2755 samples
Epoch 1/5
30160/30160 [==============================] - 62s 2ms/step - loss: 0.7218 - acc: 0.8688 - f1: 0.8060 - val_loss: 0.3959 - val_acc: 0.8715 - val_f1: 0.6481
Epoch 2/5
30160/30160 [==============================] - 57s 2ms/step - loss: 0.5201 - acc: 0.9143 - f1: 0.8655 - val_loss: 0.2219 - val_acc: 0.9162 - val_f1: 0.7051
Epoch 3/5
30160/30160 [==============================] - 52s 2ms/step - loss: 0.4326 - acc: 0.9263 - f1: 0.8834 - val_loss: 0.1821 - val_acc: 0.9379 - val_f1: 0.7358
Epoch 4/5
30160/30160 [==============================] - 52s 2ms/step - loss: 0.3627 - acc: 0.9372 - f1: 0.8999 - val_loss: 0.3820 - val_acc: 0.8505 - val_f1: 0.6245
Epoch 5/5
30160/30160 [==============================] - 52s 2ms/step - loss: 0.2957 - acc: 0.9490 - f1: 0.9178 - val_loss: 0.2386 - val_acc: 0.9169 - val_f1: 0.6964


In [17]:
save_model(model_sad,indices=[{"indices":["turn1","turn2","turn3"],"NR_WORDS":NR_WORDS,"MAX_PROP_LENGTH":MAX_PROP_LENGTH}])

In [18]:
Y_train_happy = np.array(Y_train_happy)

model_happy = clone_model(model)
model_happy.compile(loss = 'binary_crossentropy', optimizer=rmsprop, metrics = ['accuracy', f1])
model_happy.name = "happy_model"
history = model_happy.fit(X_train, Y_train_happy,
                    validation_data=(X_test, Y_test_happy),
                    epochs=5, verbose=1, batch_size=batch_size,callbacks=[mdcheck],
                    class_weight={
                        0: len(X_train) / len(np.where(Y_train_happy[:,0]==1.0)[0]),
                        1: len(X_train) / len(np.where(Y_train_happy[:,1]==1.0)[0]),
                    })

Train on 30160 samples, validate on 2755 samples
Epoch 1/5
30160/30160 [==============================] - 61s 2ms/step - loss: 0.7524 - acc: 0.8568 - f1: 0.7652 - val_loss: 0.3349 - val_acc: 0.8784 - val_f1: 0.6413
Epoch 2/5
30160/30160 [==============================] - 54s 2ms/step - loss: 0.5562 - acc: 0.9026 - f1: 0.8266 - val_loss: 0.5088 - val_acc: 0.7633 - val_f1: 0.5568
Epoch 3/5
30160/30160 [==============================] - 53s 2ms/step - loss: 0.4633 - acc: 0.9122 - f1: 0.8433 - val_loss: 0.5003 - val_acc: 0.7964 - val_f1: 0.5783
Epoch 4/5
30160/30160 [==============================] - 54s 2ms/step - loss: 0.3988 - acc: 0.9245 - f1: 0.8632 - val_loss: 0.5192 - val_acc: 0.7869 - val_f1: 0.5693
Epoch 5/5
30160/30160 [==============================] - 67s 2ms/step - loss: 0.3437 - acc: 0.9334 - f1: 0.8770 - val_loss: 0.5011 - val_acc: 0.7971 - val_f1: 0.5792


In [19]:
save_model(model_happy,indices=[{"indices":["turn1","turn2","turn3"],"NR_WORDS":NR_WORDS,"MAX_PROP_LENGTH":MAX_PROP_LENGTH}])

In [20]:
Y_train_others = np.array(Y_train_others)

model_others = clone_model(model)
model_others.compile(loss = 'binary_crossentropy', optimizer=rmsprop, metrics = ['accuracy', f1])
model_others.name = "others_model"
history = model_others.fit(X_train, Y_train_others,
                    validation_data=(X_test, Y_test_others),
                    epochs=5, verbose=1, batch_size=batch_size,callbacks=[mdcheck],
                    class_weight={
                        0: len(X_train) / len(np.where(Y_train_others[:,0]==1.0)[0]),
                        1: len(X_train) / len(np.where(Y_train_others[:,1]==1.0)[0]),
                    })

Train on 30160 samples, validate on 2755 samples
Epoch 1/5
30160/30160 [==============================] - 76s 3ms/step - loss: 0.8209 - acc: 0.8167 - f1: 0.8132 - val_loss: 0.3435 - val_acc: 0.8606 - val_f1: 0.7620
Epoch 2/5
30160/30160 [==============================] - 46s 2ms/step - loss: 0.6428 - acc: 0.8663 - f1: 0.8655 - val_loss: 0.4399 - val_acc: 0.8279 - val_f1: 0.7322
Epoch 3/5
30160/30160 [==============================] - 46s 2ms/step - loss: 0.5564 - acc: 0.8888 - f1: 0.8878 - val_loss: 0.4147 - val_acc: 0.8192 - val_f1: 0.7261
Epoch 4/5
30160/30160 [==============================] - 49s 2ms/step - loss: 0.4764 - acc: 0.9070 - f1: 0.9063 - val_loss: 0.4662 - val_acc: 0.8196 - val_f1: 0.7253
Epoch 5/5
30160/30160 [==============================] - 53s 2ms/step - loss: 0.4021 - acc: 0.9216 - f1: 0.9210 - val_loss: 0.5352 - val_acc: 0.8069 - val_f1: 0.7135


In [21]:
save_model(model_others,indices=[{"indices":["turn1","turn2","turn3"],"NR_WORDS":NR_WORDS,"MAX_PROP_LENGTH":MAX_PROP_LENGTH}])